### Docker Command Output: `docker exec -it redpanda-1 rpk version`


Version:     v24.2.18
Git ref:     f9a22d4430
Build date:  2025-02-14T12:52:55Z
OS/Arch:     linux/amd64
Go version:  go1.23.1

Redpanda Cluster
  node-1  v24.2.18 - f9a22d443087b824803638623d6b7492ec8221f9

### Docker Command Output: `docker exec -it redpanda-1 rpk topic create green-trips`


TOPIC        STATUS
green-trips  OK

In [4]:
import kafka
kafka.__version__

'2.1.1'

In [1]:
import json

from kafka import KafkaProducer

def json_serializer(data):
    return json.dumps(data).encode('utf-8')

server = 'localhost:9092'

producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=json_serializer
)

producer.bootstrap_connected()

True

In [3]:
import pandas as pd
import json
from kafka import KafkaProducer
from time import time

# Define Kafka topic and server
topic_name = "green-trips"
server = "localhost:9092"

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers=[server],
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Read CSV file
file_path = "green_tripdata_2019-10.csv"
df = pd.read_csv(file_path, usecols=[
    'lpep_pickup_datetime', 'lpep_dropoff_datetime', 
    'PULocationID', 'DOLocationID', 
    'passenger_count', 'trip_distance', 'tip_amount'
])

# Convert timestamp columns to string (JSON doesn't support Timestamp types)
df['lpep_pickup_datetime'] = df['lpep_pickup_datetime'].astype(str)
df['lpep_dropoff_datetime'] = df['lpep_dropoff_datetime'].astype(str)

# Start measuring time
t0 = time()

# Send each row as a Kafka message
for _, row in df.iterrows():
    message = row.to_dict()
    producer.send(topic_name, value=message)

# Flush producer to ensure all messages are sent
producer.flush()

# Stop measuring time
t1 = time()
took = t1 - t0

# Output total time taken
print(f"Time taken to send dataset: {took:.2f} seconds")


Time taken to send dataset: 214.62 seconds


In [5]:
from kafka import KafkaProducer
import pandas as pd
import json
from time import time

# Kafka settings
topic_name = "green-trips"
bootstrap_servers = "localhost:9092"  # Change this if Kafka runs on another server

# Initialize Kafka producer
producer = KafkaProducer(
    bootstrap_servers=bootstrap_servers,
    value_serializer=lambda v: json.dumps(v).encode('utf-8')
)

# Read CSV file (Assume 'green_tripdata.csv' is the dataset)
df = pd.read_csv("green_tripdata_2019-10.csv", usecols=[
    'lpep_pickup_datetime', 'lpep_dropoff_datetime',
    'PULocationID', 'DOLocationID', 'passenger_count',
    'trip_distance', 'tip_amount'
])

# Start timing
t0 = time()

# Send each row as a message
for _, row in df.iterrows():
    message = row.to_dict()
    producer.send(topic_name, value=message)

# Flush to ensure all messages are sent
producer.flush()

# Stop timing
t1 = time()
took = t1 - t0

print(f"Time taken to send all messages and flush: {took:.2f} seconds")

Time taken to send all messages and flush: 167.20 seconds


In [1]:
from pyflink.datastream import StreamExecutionEnvironment
from pyflink.table import EnvironmentSettings, StreamTableEnvironment

# ✅ 1️⃣ Set up the execution environment
env = StreamExecutionEnvironment.get_execution_environment()
env.enable_checkpointing(10 * 1000)  # Enable checkpoints every 10 seconds
env.set_parallelism(3)

settings = EnvironmentSettings.new_instance().in_streaming_mode().build()
t_env = StreamTableEnvironment.create(env, environment_settings=settings)

# ✅ 2️⃣ Define the source Kafka table (`green-trips`)
t_env.execute_sql("""
    CREATE TABLE green_trips (
        lpep_pickup_datetime TIMESTAMP(3),
        lpep_dropoff_datetime TIMESTAMP(3),
        PULocationID INT,
        DOLocationID INT,
        passenger_count INT,
        trip_distance DOUBLE,
        tip_amount DOUBLE,
        event_watermark AS lpep_dropoff_datetime, 
        WATERMARK FOR event_watermark AS event_watermark - INTERVAL '5' SECOND
    ) WITH (
        'connector' = 'kafka',
        'topic' = 'green-trips',
        'properties.bootstrap.servers' = 'redpanda-1:29092',
        'properties.auto.offset.reset' = 'earliest',
        'format' = 'json'
    )
""")

# ✅ 3️⃣ Define the sink Postgres table (`longest_trip_streak`)
t_env.execute_sql("""
    CREATE TABLE longest_trip_streak (
        pickup_location INT,
        dropoff_location INT,
        longest_streak BIGINT,
        PRIMARY KEY (pickup_location, dropoff_location) NOT ENFORCED
    ) WITH (
        'connector' = 'jdbc',
        'url' = 'jdbc:postgresql://postgres:5432/postgres',
        'table-name' = 'longest_trip_streak',
        'username' = 'postgres',
        'password' = 'postgres',
        'driver' = 'org.postgresql.Driver'
    )
""")

# ✅ 4️⃣ Define the **session window** aggregation query
query = """
    INSERT INTO longest_trip_streak
    SELECT 
        PULocationID AS pickup_location,
        DOLocationID AS dropoff_location,
        COUNT(*) AS longest_streak
    FROM TABLE(
        SESSION(TABLE green_trips, 
                DESCRIPTOR(event_watermark), 
                INTERVAL '5' MINUTE)
    )
    GROUP BY PULocationID, DOLocationID;
"""

# ✅ 5️⃣ Execute the query
t_env.execute_sql(query).wait()

print("✅ Session Window Job Completed!")


Py4JJavaError: An error occurred while calling o20.executeSql.
: java.lang.IllegalStateException: Property group.id is required when using committed offset for offsets initializer
	at org.apache.flink.util.Preconditions.checkState(Preconditions.java:193)
	at org.apache.flink.connector.kafka.source.enumerator.initializer.ReaderHandledOffsetsInitializer.validate(ReaderHandledOffsetsInitializer.java:75)
	at org.apache.flink.connector.kafka.source.KafkaSourceBuilder.sanityCheck(KafkaSourceBuilder.java:531)
	at org.apache.flink.connector.kafka.source.KafkaSourceBuilder.build(KafkaSourceBuilder.java:432)
	at org.apache.flink.streaming.connectors.kafka.table.KafkaDynamicSource.createKafkaSource(KafkaDynamicSource.java:493)
	at org.apache.flink.streaming.connectors.kafka.table.KafkaDynamicSource.getScanRuntimeProvider(KafkaDynamicSource.java:250)
	at org.apache.flink.table.planner.plan.nodes.exec.common.CommonExecTableSourceScan.translateToPlanInternal(CommonExecTableSourceScan.java:121